<a href="https://colab.research.google.com/github/steveola/niya_ml_data/blob/main/Module_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import our essential toolkits
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load our dataset
df = pd.read_csv('student_data.csv')
print("Data Loaded Successfully!")
print(df.head())

# X is our Feature Matrix (everything we use to make a prediction)
X = df[['Hours_Studied', 'Previous_Grade']]
# y is our Label Vector (the thing we want to predict)
y = df['Pass']
print("Features (X):")
print(X.head())
print("\nLabels (y):")
print(y.head())

# Split the data: 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {X_train.shape[0]} students")
print(f"Testing set size: {X_test.shape[0]} students")

# Create a Decision Tree classifier
model = DecisionTreeClassifier(random_state=42)
# TRAIN THE MODEL! This is the magic line.
model.fit(X_train, y_train)
print("Model training complete!")


# Use the trained model to make predictions on the test set
y_pred = model.predict(X_test)
print("Predictions for the test set:")
print(y_pred)


# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f} ({accuracy*100:.0f}%)")
# Create a Confusion Matrix to see WHERE we went wrong
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)
# Let's make a visual heatmap of the confusion matrix
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
xticklabels=['Predicted Fail', 'Predicted Pass'],
yticklabels=['Actual Fail', 'Actual Pass'])
plt.title('Confusion Matrix for our Student Pass/Fail Predictor')
plt.show()

# Let's try a less complex tree
better_model = DecisionTreeClassifier(max_depth=3, random_state=42)
better_model.fit(X_train, y_train)
better_pred = better_model.predict(X_test)
print(f"Improved Model Accuracy: {accuracy_score(y_test, better_pred):.2f}")


# Step 1: Import Libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Step 2: Load and Prepare the Data (using our preprocessed student data)
# Let's assume X contains ['Hours_Studied', 'Previous_Grade'] and y is 'Pass' (1 or 0)
X = df[['Hours_Studied', 'Previous_Grade']].values
y = df['Pass'].values
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# It's CRUCIAL to scale data for neural networks!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 3: Build the Neural Network Model
model = keras.Sequential([
# Input Layer: 2 features, so 2 inputs. Keras figures this out automatically.
# First Hidden Layer: 8 neurons, ReLU activation.
keras.layers.Dense(8, activation='relu', input_shape=(2,)),
# Second Hidden Layer: 4 neurons, ReLU activation.
keras.layers.Dense(4, activation='relu'),
# Output Layer: 1 neuron (for binary classification), Sigmoid activation.
keras.layers.Dense(1, activation='sigmoid')
])
# Let's look at our creation!
print("Model Architecture:")
model.summary()

# Step 4: Compile the Model (Configure the learning process)
model.compile(optimizer='adam', # A powerful and popular optimizer
loss='binary_crossentropy', # The loss function for binary classification
metrics=['accuracy']) # What to monitor during training
print("Model compiled and ready for training!")
"The moment of truth... let's train it!"


# Step 5: Train the Model (The "Fitting" process)
# Number of passes through the training data
# Check performance on test set after each epoch
history = model.fit(X_train_scaled, y_train,epochs=50,
validation_data=(X_test_scaled, y_test), verbose=1) # Shows a progress bar
print("Training complete!")

# Step 6: Evaluate the Final Model
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")
# Step 7: Make a Prediction
new_student = np.array([[12, 80]]) # A student who studied 12 hours with an 80 previousm grade
new_student_scaled = scaler.transform(new_student)
prediction_prob = model.predict(new_student_scaled)
print(f"Probability of passing: {prediction_prob[0][0]:.4f}")


# First, let's re-train our best model from Module 8 to make sure we have it
from sklearn.tree import DecisionTreeClassifier
import joblib
# Assume X_train and y_train are already defined
final_model = DecisionTreeClassifier(max_depth=3, random_state=42)
final_model.fit(X_train, y_train)
# Now, let's save it using joblib
joblib.dump(final_model, 'my_student_predictor.joblib')
print("Model saved successfully! Check your Colab files folder on the left.")

# Imagine we are in a brand new notebook or a different app.
# We can load the model back into memory
loaded_model = joblib.load('my_student_predictor.joblib')
# And use it to make a prediction just like before!
new_student = [[10, 75]] # A student who studied 10 hours with a previous grade of 75%
prediction = loaded_model.predict(new_student)
print(f"This student is predicted to: {'PASS' if prediction[0] == 1 else 'FAIL'}")


# This is a simplified example. Running a full Flask app in Colab requires extra steps.
from flask import Flask, request, jsonify
import joblib
# Load our saved model
model = joblib.load('my_student_predictor.joblib')
app = Flask(__name__)
@app.route('/predict', methods=['POST'])
def predict():
# Get the JSON data from the request
    data = request.get_json()
    # Extract the features from the data
    hours_studied = data['hours_studied']
    previous_grade = data['previous_grade']
    # Make a prediction
    prediction = model.predict([[hours_studied, previous_grade]])
    # Return the prediction as a JSON response
    result = {'prediction': 'PASS' if prediction[0] == 1 else 'FAIL'}
    return jsonify(result)

# This would run the server if we were on a local machine
# if __name__ == '__main__':
# app.run(debug=True)

import streamlit as st
import joblib
# Load the model
model = joblib.load('my_student_predictor.joblib')
# Create the web app
st.title(" Student Success Predictor")
st.write("Will a student pass their final exam?")
# Create input widgets
hours = st.slider("Hours Studied", 0, 20, 10)
previous_grade = st.slider("Previous Grade (%)", 0, 100, 75)

# When the user clicks the button, make a prediction
if st.button('Predict'):
    prediction = model.predict([[hours, previous_grade]])
    result = 'PASS' if prediction[0] == 1 else 'FAIL'
    st.success(f"This student is predicted to: {result}")




